In [ ]:
import pandas as pd
import glob
import os
import unicodedata
import re

pd.options.display.max_columns = None

In [2]:
# Ruta relativa desde la carpeta scripts
folder_path = '../data/input/Betplay/'

# Obtener lista de archivos Excel
excel_files = glob.glob(os.path.join(folder_path, '*.xlsx'))

# Leer todos los archivos
dfs = [pd.read_excel(file) for file in excel_files]

# Concatenar todos los DataFrames en uno
combined_df = pd.concat(dfs, ignore_index=True)

# Ver ejemplo
print(f'{len(dfs)} archivos leídos')

120 archivos leídos


In [45]:
df = combined_df.copy()

df = df.dropna(subset=['Partido'])

def rename_columns_with_slash(columns):
    new_columns = []
    i = 0
    while i < len(columns):
        col = columns[i]

        # Si tiene "/", expandimos usando el número de partes
        if "/" in col:
            parts = [p.strip() for p in col.split("/")]
            base = parts[0]
            for j in range(len(parts)):
                if i + j < len(columns):
                    name = base if j == 0 else f"{base} {parts[j]}"
                    new_columns.append(name)
            i += len(parts)
        # Si es 'Unnamed', intenta continuar desde antes
        elif "Unnamed" in col:
            # Si no hemos expandido antes, darle nombre genérico
            new_columns.append(f"%_{parts[0]}")
            i += 1
        else:
            new_columns.append(col)
            i += 1

    # Rellenar si quedaron columnas faltantes
    while len(new_columns) < len(columns):
        new_columns.append(f"col_{len(new_columns)}")

    return new_columns

# Aplicar al DataFrame
df.columns = rename_columns_with_slash(list(df.columns))

df['Fecha'].value_counts().sort_index(ascending=False)
df['Competición'].value_counts().sort_index(ascending=False)

df = df[df['Competición'] == 'Colombia. Liga BetPlay']

df = df[df['Fecha'] >= '2025-01-01']

df.drop_duplicates(inplace=True)

print(df.shape)

df.head(2)

(426, 109)


,Fecha,Partido,Competición,Duración,Equipo,Seleccionar esquema,Goles,xG,Tiros,Tiros a la portería,%_Tiros,Pases,Pases logrados,%_Pases,"Posesión del balón, %",Balones perdidos,Balones perdidos bajos,Balones perdidos medios,Balones perdidos altos,Balones recuperados,Balones recuperados bajos,Balones recuperados medios,Balones recuperados altos,Duelos,Duelos ganados,%_Duelos,Tiros de fuera del área,Tiros de fuera del área a la portería,%_Tiros de fuera del área,Ataques posicionales,Ataques posicionales con remate,%_Ataques posicionales,Contraataques,Contraataques con remate,%_Contraataques,Jugadas a balón parado,Jugadas a balón parado con remate,%_Jugadas a balón parado,Córneres,Córneres con remate,%_Córneres,Tiros libres,Tiros libres con remate,%_Tiros libres,Penaltis,Penaltis marcados,%_Penaltis,Centros,Centros precisos,%_Centros,Pases cruzados en profundidad completados,Pases en profundidad completados,Entradas al área de penalti (carreras,Entradas al área de penalti (carreras pases cruzados),%_Entradas al área de penalti (carreras,Toques en el área de penalti,Duelos ofensivos,Duelos ofensivos ganados,%_Duelos ofensivos,Fuera de juego,Goles recibidos,Tiros en contra,Tiros en contra a la portería,%_Tiros en contra,Duelos defensivos,Duelos defensivos ganados,%_Duelos defensivos,Duelos aéreos,Duelos aéreos ganados,%_Duelos aéreos,Entradas a ras de suelo,Entradas a ras de suelo logradas,%_Entradas a ras de suelo,Interceptaciones,Despejes,Faltas,Tarjetas amarillas,Tarjetas rojas,Pases hacia adelante,Pases hacia adelante logrados,%_Pases hacia adelante,Pases hacia atrás,Pases hacia atrás logrados,%_Pases hacia atrás,Pases laterales,Pases laterales logrados,%_Pases laterales,Pases largos,Pases largos logrados,%_Pases largos,Pases en el último tercio,Pases en el último tercio logrados,%_Pases en el último tercio,Pases progresivos,Pases progresivos precisos,%_Pases progresivos,Desmarques,Desmarques logrados,%_Desmarques,Saques laterales,Saques laterales logrados,%_Saques laterales,Saques de meta,Intensidad de paso,Promedio pases por posesión del balón,Lanzamiento largo %,Distancia media de tiro,Longitud media pases,PPDA
438,2025-05-25,Alianza - Santa Fe 6:1,Colombia. Liga BetPlay,96.0,Alianza,4-2-3-1 (100.0%),6.0,1.72,13.0,6.0,46.15,408.0,349.0,85.54,56.99,81.0,14.0,28.0,39.0,66.0,34.0,24.0,8.0,138.0,73.0,52.90,6.0,1.0,16.67,26.0,7.0,26.92,0.0,0.0,0.0,18.0,4.0,22.22,3.0,1.0,33.33,3.0,1.0,33.33,0.0,0.0,0.0,14.0,6.0,42.86,5.0,3.0,18.0,3.0,6.0,16.0,51.0,22.0,43.14,3.0,1.0,8.0,3.0,37.50,49.0,32.0,65.31,26.0,15.0,57.69,4.0,1.0,25.0,30.0,14.0,13.0,0.0,0.0,136.0,102.0,75.00,70.0,68.0,97.14,129.0,116.0,89.92,42.0,21.0,50.00,39.0,24.0,61.54,61.0,40.0,65.57,2.0,1.0,50.00,10.0,10.0,100.00,4.0,16.49,4.48,10.29,17.9,18.02,9.65
439,2025-05-25,Alianza - Santa Fe 6:1,Colombia. Liga BetPlay,96.0,Santa Fe,4-3-3 (100.0%),1.0,0.46,8.0,3.0,37.50,293.0,234.0,79.86,43.01,95.0,10.0,34.0,51.0,55.0,23.0,24.0,8.0,138.0,63.0,45.65,6.0,2.0,33.33,22.0,5.0,22.73,2.0,0.0,0.0,26.0,3.0,11.54,4.0,2.0,50.00,6.0,0.0,0.00,0.0,0.0,0.0,9.0,2.0,22.22,2.0,5.0,11.0,3.0,3.0,10.0,49.0,17.0,34.69,1.0,6.0,13.0,7.0,53.85,51.0,29.0,56.86,26.0,9.0,34.62,1.0,1.0,100.0,41.0,10.0,11.0,2.0,0.0,89.0,53.0,59.55,32.0,31.0,96.88,132.0,115.0,87.12,30.0,16.0,53.33,28.0,14.0,50.00,45.0,31.0,68.89,3.0,1.0,33.33,18.0,17.0,94.44,7.0,15.69,3.22,10.24,20.1,19.77,9.28


In [ ]:
# df.head(2).to_csv('../data/output/combined_betplay.csv', index=False)

In [46]:
# 1. Columnas identificadoras y métricas
id_cols = ['Fecha', 'Partido', 'Equipo']
metric_cols = [col for col in df.columns if col not in id_cols]

# 2. Preparar el DataFrame contrario
df_contra = df.copy()
df_contra = df_contra.rename(
    columns={**{col: f"{col}_contra" for col in metric_cols}, 'Equipo': 'Equipo_contra'}
)

# 3. Hacer merge por Fecha y Partido (equipo contrario)
df_full = df.merge(
    df_contra,
    on=['Fecha', 'Partido'],
    suffixes=('', '_drop')
)

# 4. Filtrar para evitar unir el equipo consigo mismo
df_full = df_full[df_full['Equipo'] != df_full['Equipo_contra']]

# 5. Eliminar columnas "_drop" si se generaron
df_full = df_full.drop(columns=[col for col in df_full.columns if col.endswith('_drop')])

# 6. Resetear índice
df_full = df_full.reset_index(drop=True)

df_full.drop_duplicates(inplace=True)

df_full.shape

# Detectar si el Equipo es el que inicia el nombre del Partido → Local
df_full['Tipo'] = df_full.apply(
    lambda row: 'Local' if row['Partido'].startswith(row['Equipo']) else 'Visitante',
    axis=1
)


In [47]:
def limpiar_columna(col):
    # Convertir a minúsculas
    col = col.lower()
    # Eliminar tildes
    col = unicodedata.normalize('NFKD', col).encode('ascii', 'ignore').decode('utf-8')
    # Reemplazar espacios por guión bajo
    col = col.replace(" ", "_")
    # Quitar caracteres no alfanuméricos ni _
    col = re.sub(r"[^\w_]", "", col)
    # Eliminar guiones bajos al inicio
    col = re.sub(r"^_+", "%", col)
    return col

# Aplicar a todas las columnas
df_full.columns = [limpiar_columna(col) for col in df_full.columns]


In [48]:
# df_kk = df_full[df_full['Equipo'] == 'Alianza']

# Ordenar por equipo y fecha
df_full.sort_values(by=["equipo", "fecha"], inplace=True)

# Crear columna 'jornada'
df_full["jornada"] = df_full.groupby("equipo").cumcount() + 1

df_full.to_excel('../data/output/combined_betplay.xlsx', index=False)
df_full.to_csv('../data/output/combined_betplay.csv', index=False)
# df_kk[['Fecha', 'Partido', 'Equipo', 'Equipo_contra', 'Tiros','Tiros_contra','xG_contra', 'xG']]

# df_kk['xG_contra'].mean()

In [40]:
# Suma
df_sum_div = df_full.groupby(['equipo', 'tipo'], as_index=False).sum(numeric_only=True)

# Promedio
df_mean_div = df_full.groupby(['equipo', 'tipo'], as_index=False).mean(numeric_only=True)

# Copiar suma
df_per90_div = df_sum_div.copy()

# Columnas a ajustar (todas menos claves)
cols = [col for col in df_per90_div.columns if col not in ['equipo', 'tipo', 'duracion']]

# Duración total por grupo
duracion = df_per90_div['duracion']

# Calcular métricas normalizadas
for col in cols:
    df_per90_div[col] = (90 / duracion) * df_per90_div[col]
    df_per90_div.rename(columns={col: f"{col}_per90"}, inplace=True)

In [41]:
# Agrupaciones base
df_sum = df_full.groupby("equipo", as_index=False).sum(numeric_only=True)
df_mean = df_full.groupby("equipo", as_index=False).mean(numeric_only=True)

# Copiar df_sum para trabajar sobre él
df_per90 = df_sum.copy()

# Guardar y eliminar duración para el cálculo
duracion_total = df_sum['duracion']

# Aplicar fórmula por columna numérica (excepto 'duracion')
cols_to_adjust = [col for col in df_per90.columns if col not in ['equipo', 'duracion']]

for col in cols_to_adjust:
    df_per90[col] = (90 / duracion_total) * df_per90[col]


In [42]:
df_per90[df_per90['equipo']=='Alianza'].head(2)

,equipo,duracion,goles,xg,tiros,tiros_a_la_porteria,%tiros,pases,pases_logrados,%pases,posesion_del_balon_,balones_perdidos,balones_perdidos_bajos,balones_perdidos_medios,balones_perdidos_altos,balones_recuperados,balones_recuperados_bajos,balones_recuperados_medios,balones_recuperados_altos,duelos,duelos_ganados,%duelos,tiros_de_fuera_del_area,tiros_de_fuera_del_area_a_la_porteria,%tiros_de_fuera_del_area,ataques_posicionales,ataques_posicionales_con_remate,%ataques_posicionales,contraataques,contraataques_con_remate,%contraataques,jugadas_a_balon_parado,jugadas_a_balon_parado_con_remate,%jugadas_a_balon_parado,corneres,corneres_con_remate,%corneres,tiros_libres,tiros_libres_con_remate,%tiros_libres,penaltis,penaltis_marcados,%penaltis,centros,centros_precisos,%centros,pases_cruzados_en_profundidad_completados,pases_en_profundidad_completados,entradas_al_area_de_penalti_carreras,entradas_al_area_de_penalti_carreras_pases_cruzados,%entradas_al_area_de_penalti_carreras,toques_en_el_area_de_penalti,duelos_ofensivos,duelos_ofensivos_ganados,%duelos_ofensivos,fuera_de_juego,goles_recibidos,tiros_en_contra,tiros_en_contra_a_la_porteria,%tiros_en_contra,duelos_defensivos,duelos_defensivos_ganados,%duelos_defensivos,duelos_aereos,duelos_aereos_ganados,%duelos_aereos,entradas_a_ras_de_suelo,entradas_a_ras_de_suelo_logradas,%entradas_a_ras_de_suelo,interceptaciones,despejes,faltas,tarjetas_amarillas,tarjetas_rojas,pases_hacia_adelante,pases_hacia_adelante_logrados,%pases_hacia_adelante,pases_hacia_atras,pases_hacia_atras_logrados,%pases_hacia_atras,pases_laterales,pases_laterales_logrados,%pases_laterales,pases_largos,pases_largos_logrados,%pases_largos,pases_en_el_ultimo_tercio,pases_en_el_ultimo_tercio_logrados,%pases_en_el_ultimo_tercio,pases_progresivos,pases_progresivos_precisos,%pases_progresivos,desmarques,desmarques_logrados,%desmarques,saques_laterales,saques_laterales_logrados,%saques_laterales,saques_de_meta,intensidad_de_paso,promedio_pases_por_posesion_del_balon,lanzamiento_largo_,distancia_media_de_tiro,longitud_media_pases,ppda,duracion_contra,goles_contra,xg_contra,tiros_contra,tiros_a_la_porteria_contra,%tiros_contra,pases_contra,pases_logrados_contra,%pases_contra,posesion_del_balon__contra,balones_perdidos_contra,balones_perdidos_bajos_contra,balones_perdidos_medios_contra,balones_perdidos_altos_contra,balones_recuperados_contra,balones_recuperados_bajos_contra,balones_recuperados_medios_contra,balones_recuperados_altos_contra,duelos_contra,duelos_ganados_contra,%duelos_contra,tiros_de_fuera_del_area_contra,tiros_de_fuera_del_area_a_la_porteria_contra,%tiros_de_fuera_del_area_contra,ataques_posicionales_contra,ataques_posicionales_con_remate_contra,%ataques_posicionales_contra,contraataques_contra,contraataques_con_remate_contra,%contraataques_contra,jugadas_a_balon_parado_contra,jugadas_a_balon_parado_con_remate_contra,%jugadas_a_balon_parado_contra,corneres_contra,corneres_con_remate_contra,%corneres_contra,tiros_libres_contra,tiros_libres_con_remate_contra,%tiros_libres_contra,penaltis_contra,penaltis_marcados_contra,%penaltis_contra,centros_contra,centros_precisos_contra,%centros_contra,pases_cruzados_en_profundidad_completados_contra,pases_en_profundidad_completados_contra,entradas_al_area_de_penalti_carreras_contra,entradas_al_area_de_penalti_carreras_pases_cruzados_contra,%entradas_al_area_de_penalti_carreras_contra,toques_en_el_area_de_penalti_contra,duelos_ofensivos_contra,duelos_ofensivos_ganados_contra,%duelos_ofensivos_contra,fuera_de_juego_contra,goles_recibidos_contra,tiros_en_contra_contra,tiros_en_contra_a_la_porteria_contra,%tiros_en_contra_contra,duelos_defensivos_contra,duelos_defensivos_ganados_contra,%duelos_defensivos_contra,duelos_aereos_contra,duelos_aereos_ganados_contra,%duelos_aereos_contra,entradas_a_ras_de_suelo_contra,entradas_a_ras_de_suelo_logradas_contra,%entradas_a_ras_de_suelo_contra,interceptaciones_contra,despejes_contra,faltas_contra,tarjetas_amarillas_contra,tarjetas_rojas_contr

In [43]:
# 1. Filtrar las columnas necesarias
df_xg = df_full[["fecha", "equipo", "xg", "xg_contra"]].copy()

# 2. Ordenar por Equipo y Fecha (ascendente)
df_xg.sort_values(by=["equipo", "fecha"], inplace=True)

# 3. Enumerar por equipo (índice consecutivo por grupo)
df_xg["jornada"] = df_xg.groupby("equipo").cumcount() + 1

# 4. Eliminar la columna Fecha
df_xg.drop(columns=["fecha"], inplace=True)

# 5. (Opcional) Reordenar columnas si deseas
df_xg = df_xg[["equipo", "jornada", "xg", "xg_contra"]]


In [44]:
df_xg

# df_kk

,equipo,jornada,xg,xg_contra
36,Alianza,1,0.50,0.63
34,Alianza,2,0.88,0.54
32,Alianza,3,1.20,1.70
30,Alianza,4,0.33,2.05
28,Alianza,5,1.48,1.53
...,...,...,...,...
57,Águilas Doradas,15,0.36,0.92
249,Águilas Doradas,16,2.95,1.73
273,Águilas Doradas,17,1.12,1.44
133,Águilas Doradas,18,1.38,1.24
